In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
#PATH = "C:\Users\mcveigh\Documents\PythonPC\chrome.exe"
#driver = webdriver.Chrome(PATH)
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import re
import os
import sys
import bs4
from bs4 import BeautifulSoup
import requests
import numpy as np

### List of URLs input to search short list manually entered

In [ ]:
urls = ['https://www.microbiologyresearch.org/content/journal/ijsem/10.1099/ijsem.0.006404',
       'https://www.microbiologyresearch.org/content/journal/ijsem/10.1099/ijsem.0.006406?emailalert=true',
        'https://www.microbiologyresearch.org/content/journal/ijsem/10.1099/ijsem.0.006401?emailalert=true',
        'https://www.microbiologyresearch.org/content/journal/ijsem/10.1099/ijsem.0.006417?emailalert=true'
       ]

### Input URLS from saved email in html - save as from outlook in htm format

In [2]:
input = (r'IJSEMemail2.htm')

In [3]:
#with open ('IJSEMemail1.htm', encoding = 'unicode_escape') as f:
with open (input, encoding = 'unicode_escape') as f:
    content = f.read()
    soup = BeautifulSoup(content, 'html.parser')  
print(soup.prettify())

<html xmlns="http://www.w3.org/TR/REC-html40" xmlns:m="http://schemas.microsoft.com/office/2004/12/omml" xmlns:ns0="http://www.w3.org/1999/xhtml" xmlns:o="urn:schemas-microsoft-com:office:office" xmlns:v="urn:schemas-microsoft-com:vml" xmlns:w="urn:schemas-microsoft-com:office:word">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="Word.Document" name="ProgId"/>
  <meta content="Microsoft Word 15" name="Generator"/>
  <meta content="Microsoft Word 15" name="Originator"/>
  <link href="IJSEMemail2_files/filelist.xml" rel="File-List"/>
  <link href="IJSEMemail2_files/editdata.mso" rel="Edit-Time-Data"/>
  <!--[if !mso]>
<style>
v\:* {behavior:url(#default#VML);}
o\:* {behavior:url(#default#VML);}
w\:* {behavior:url(#default#VML);}
.shape {behavior:url(#default#VML);}
</style>
<![endif]-->
  <!--[if gte mso 9]><xml>
 <o:OfficeDocumentSettings>
  <o:AllowPNG/>
 </o:OfficeDocumentSettings>
</xml><![endif]-->
  <link href="IJSEMemail2_files/them

In [4]:
urls = []
numberoflinks = 0
for url in soup.findAll( 'a', attrs={'href': re.compile("^https:")}):
    urls.append(url.get('href'))
    numberoflinks = numberoflinks + 1
#print(numberoflinks)
#print(urls)
#remove_list = ['TandC','doi.org', 'myaccount']
urls = [e for e in urls if "TandC" not in e and "doi.org" not in e and "myaccount" not in e]
#print(urls)

### Selenium web scraper - title and author note (footnote 1)

In [13]:
#Function to scrape headlines using Selenium
def scrape_with_selenium(url, counter):
    global title
    global footnote
    global description
    title = []
    description = None
    options = Options()
    options.headless = False  # Set to True for headless mode
    driver = webdriver.Chrome(options=options)

    #Navigate to the webpage
    driver.get(url)

    #Allow time for dynamic content to load (you may need to use WebDriverWait for more robust waiting)
    time.sleep(3)

    for element in driver.find_elements(By.CLASS_NAME, "item-meta-data__item-title"):
        #print(element.text)
        title = element.text
        #print(title)
    #for element in driver.find_elements(By.CLASS_NAME, "author-footnote-below-abstract"):get.text() #retrieves all footnotes
    for element in driver.find_elements(By.ID, "FN1"): #retrieves just footnote 1
        #print(element.text)
        footnote = element.text
        #print(footnote)
        #print(type(footnote))
    
    for element in driver.find_elements(By.CSS_SELECTOR, "div.tl-main-part.title"): #finds section headers
        #print(element.text)
        counter += 1
        description = element.text
        print(description)
        if "Description of" in description: 
            print('found', description)  
            #snumber = 's' + str(counter - 4) + '/p[3]'
            snumber = 's' + str(counter - 4)
            print('snumber is', snumber)
            for element in driver.find_elements(By.ID, snumber):
                description = element.text
                #print(description)
        
    #Close the browser window
    driver.quit()
    return title, footnote, description



### Beautifulsoup webscraper - returns abstract

In [14]:
def scrape_with_beautifulsoup(url):
    global abstract_text
    abstract_text = None
    response = requests.get(url)
    #soup = BeautifulSoup(response.content, "html.parser")
    soup = BeautifulSoup(response.text, "lxml")
    abstract = soup.find(class_= "articleabstract")
    if abstract is not None:
    #print(abstract.get_text())
        abstract_text = abstract.get_text()
        return abstract_text
    else:
        return

### Main body 

In [18]:
pub_df = pd.DataFrame(columns=['PublishedName', 'Accessions', 'Strains','URL'])
pd.set_option('display.max_columns', None)
for url in urls:
    counter = 1
    strains = []
    accessions = []
    description = None
    #scrape_with_selenium(url, counter)
    scrape_with_beautifulsoup(url)
    options = Options()
    options.headless = False  # Set to True for headless mode
    driver = webdriver.Chrome(options=options)

    #Navigate to the webpage
    driver.get(url)

    #Allow time for dynamic content to load (you may need to use WebDriverWait for more robust waiting)
    time.sleep(3)
    
    for element in driver.find_elements(By.CLASS_NAME, "item-meta-data__item-title"):
        #print(element.text)
        title = element.text
        print(title)
        
    for element in driver.find_elements(By.CSS_SELECTOR, "div.tl-main-part.title"): #finds section headers
        #print(element.text)
        counter += 1
        description = element.text
        #print(description)
        if "Description of" in description: 
            #print('found', description)  
            #snumber = 's' + str(counter - 4) + '/p[3]'
            snumber = 's' + str(counter - 4)
            #print('snumber is', snumber)
            for element in driver.find_elements(By.ID, snumber):
                description = element.text
                #print(description)
        
                #find the organism names    
                match = [r'(\S+\s+){2}(?=sp. nov.)', r'(\S+\s+){2}(?=nom. nov.)']
                regex = re.compile(r'\b(' + '|'.join(match) + r')\b')
                orgname = [m.group() for m in regex.finditer(description)]
                print('orgname', orgname)

                #find the accessions
                pattern = [r'[A-Z]{2}\d{6}', r'[A-Z]{4}\d{8}', r'([A-Z]+)(_[A-Z]+)\d{6}', r'[A-Z]{6}\d{9}']
                regex = re.compile(r'\b(' + '|'.join(pattern) + r')\b')
                if description is not None:
                    accessions = [m.group() for m in regex.finditer(description)]
                    print('accessions', accessions)
    
                #find the strains
                #strainpattern = [r'(?<=type strain).*'] 
                strainpattern = [r'(?<=type strain).*?(?=\))']
                regex = re.compile(r'\b(' + '|'.join(strainpattern) + r')\b')
                if description is not None:
                    strains = [m.group() for m in regex.finditer(description)]
                    print('strain names', strains)
    
                #load data into pandas dataframe
                row_data = [orgname, accessions, strains, url]
                length = len(pub_df)
                pub_df.loc[length] = row_data
            print('BREAK')
    
#Close the browser window
    driver.quit()    

Proposal for reclassification of the species Hungatella xylanolytica as Lacrimispora xylanisolvens nom. nov. and transfer of the genus Hungatella to the family Lachnospiraceae
Roseateles subflavus sp. nov. and Roseateles aquae sp. nov., isolated from artificial pond water and Roseateles violae sp. nov., isolated from a Viola mandshurica root
orgname ['Roseateles subflavus ']
accessions ['JASVDS000000000', 'OQ799017']
strain names [' is APW6T (=KACC 22877T=TBRC 16606T']
orgname ['Roseateles subflavus ']
accessions []
strain names []
BREAK
orgname ['Roseateles aquae ']
accessions ['JAVXZY000000000', 'OP341190']
strain names [' is APW11T (=KACC 22878T=TBRC 16607T']
orgname ['Roseateles aquae ']
accessions []
strain names []
BREAK
orgname ['Roseateles violae ']
accessions ['JAUHHC000000000', 'OQ799020']
strain names [' is PFR6T (=KACC 23257T=TBRC 17653T']
orgname ['Roseateles violae ']
accessions []
strain names []
BREAK
Paenibacillus hexagrammi sp. nov., a novel bacterium isolated from th

In [19]:
pd.set_option('max_colwidth', None)
pub_df['Strains'] = [','.join(map(str, l)) for l in pub_df['Strains']]
pub_df

,PublishedName,Accessions,Strains,URL
0,[Roseateles subflavus ],"[JASVDS000000000, OQ799017]",is APW6T (=KACC 22877T=TBRC 16606T,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006426%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7Cede9d3e056284e72eba408dc8c64635f%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638539610385701322%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=3kRQU%2Bl1KWXJyM%2B7l1yPL2Ex%2FzlEC06BZdrpv28Vgt8%3D&reserved=0
1,[Roseateles subflavus ],[],,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006426%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7Cede9d3e056284e72eba408dc8c64635f%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638539610385701322%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=3kRQU%2Bl1KWXJyM%2B7l1yPL2Ex%2FzlEC06BZdrpv28Vgt8%3D&reserved=0
2,[Roseateles aquae ],"[JAVXZY000000000, OP341190]",is APW11T (=KACC 22878T=TBRC 16607T,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006426%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7Cede9d3e056284e72eba408dc8c64635f%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638539610385701322%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=3kRQU%2Bl1KWXJyM%2B7l1yPL2Ex%2FzlEC06BZdrpv28Vgt8%3D&reserved=0
3,[Roseateles aquae ],[],,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006426%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7Cede9d3e056284e72eba408dc8c64635f%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638539610385701322%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=3kRQU%2Bl1KWXJyM%2B7l1yPL2Ex%2FzlEC06BZdrpv28Vgt8%3D&reserved=0
4,[Roseateles violae ],"[JAUHHC000000000, OQ799020]",is PFR6T (=KACC 23257T=TBRC 17653T,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006426%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7Cede9d3e056284e72eba408dc8c64635f%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638539610385701322%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=3kRQU%2Bl1KWXJyM%2B7l1yPL2Ex%2FzlEC06BZdrpv28Vgt8%3D&reserved=0
5,[Roseateles violae ],[],,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006426%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7Cede9d3e056284e72eba408dc8c64635f%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638539610385701322%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=3kRQU%2Bl1KWXJyM%2B7l1yPL2Ex%2FzlEC06BZdrpv28Vgt8%3D&reserved=0
6,[Paenibacillus hexagrammi ],"[OM319628, CP090978, CP090979]",of Paenibacillus hexagrammi is YPD9-1T (=KCTC 43424T =LMG 32988T,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006419%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7Cede9d3e056284e72eba408dc8c64635f%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638539610385711878%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=H5hpjVJGLq1IP%2F%2B8UcDSEBrVqpIuCuYl3Q7j7YaYZxk%3D&reserved=0
7,[Paenibacillus hexagrammi ],[],,https://gcc02.safelinks.prote

In [23]:
pub_df = pub_df.drop_duplicates(subset='PublishedName', keep="first")
pub_df

,PublishedName,Accessions,Strains,URL
0,[Roseateles subflavus ],"[JASVDS000000000, OQ799017]",is APW6T (=KACC 22877T=TBRC 16606T,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006426%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7Cede9d3e056284e72eba408dc8c64635f%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638539610385701322%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=3kRQU%2Bl1KWXJyM%2B7l1yPL2Ex%2FzlEC06BZdrpv28Vgt8%3D&reserved=0
2,[Roseateles aquae ],"[JAVXZY000000000, OP341190]",is APW11T (=KACC 22878T=TBRC 16607T,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006426%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7Cede9d3e056284e72eba408dc8c64635f%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638539610385701322%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=3kRQU%2Bl1KWXJyM%2B7l1yPL2Ex%2FzlEC06BZdrpv28Vgt8%3D&reserved=0
4,[Roseateles violae ],"[JAUHHC000000000, OQ799020]",is PFR6T (=KACC 23257T=TBRC 17653T,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006426%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7Cede9d3e056284e72eba408dc8c64635f%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638539610385701322%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=3kRQU%2Bl1KWXJyM%2B7l1yPL2Ex%2FzlEC06BZdrpv28Vgt8%3D&reserved=0
6,[Paenibacillus hexagrammi ],"[OM319628, CP090978, CP090979]",of Paenibacillus hexagrammi is YPD9-1T (=KCTC 43424T =LMG 32988T,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006419%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7Cede9d3e056284e72eba408dc8c64635f%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638539610385711878%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=H5hpjVJGLq1IP%2F%2B8UcDSEBrVqpIuCuYl3Q7j7YaYZxk%3D&reserved=0
8,[Flavobacterium poyangense ],"[MH392710, JAALLN000000000]","was first isolated,, JXAS1T (=GDMCC 1.1378T=KCTC 62719T",https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006416%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7Cede9d3e056284e72eba408dc8c64635f%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638539610385722640%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=Qzrd94rIG9jH9V1Y7Vio8KHqnJN2A%2BWTRMYxBHLt780%3D&reserved=0
10,[Mangrovimonas cancribranchiae ],"[OR538380, CP136925]",", UG2_1T (=KCTC 102158T=DSM 117025T",https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006415%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7Cede9d3e056284e72eba408dc8c64635f%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638539610385733829%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=9pz15TOQ5qI7YT5FdfiMS5qM0vdHP45L1RRtx%2BhrDUo%3D&reserved=0
12,[Clostridium aquiflavi ],"[OR414528, JAVJAN000000000]",", 5 N-1T (=CICC 24886T=JCM 35355T",https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006420%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7Cede9d3e056284e72eba408dc8c64635f%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638539610385746457%7CUnknown%7CTWFpbGZsb3

In [24]:
pub_df.explode(['PublishedName']).reset_index(drop=True)

,PublishedName,Accessions,Strains,URL
0,Roseateles subflavus,"[JASVDS000000000, OQ799017]",is APW6T (=KACC 22877T=TBRC 16606T,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006426%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7Cede9d3e056284e72eba408dc8c64635f%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638539610385701322%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=3kRQU%2Bl1KWXJyM%2B7l1yPL2Ex%2FzlEC06BZdrpv28Vgt8%3D&reserved=0
1,Roseateles aquae,"[JAVXZY000000000, OP341190]",is APW11T (=KACC 22878T=TBRC 16607T,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006426%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7Cede9d3e056284e72eba408dc8c64635f%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638539610385701322%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=3kRQU%2Bl1KWXJyM%2B7l1yPL2Ex%2FzlEC06BZdrpv28Vgt8%3D&reserved=0
2,Roseateles violae,"[JAUHHC000000000, OQ799020]",is PFR6T (=KACC 23257T=TBRC 17653T,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006426%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7Cede9d3e056284e72eba408dc8c64635f%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638539610385701322%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=3kRQU%2Bl1KWXJyM%2B7l1yPL2Ex%2FzlEC06BZdrpv28Vgt8%3D&reserved=0
3,Paenibacillus hexagrammi,"[OM319628, CP090978, CP090979]",of Paenibacillus hexagrammi is YPD9-1T (=KCTC 43424T =LMG 32988T,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006419%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7Cede9d3e056284e72eba408dc8c64635f%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638539610385711878%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=H5hpjVJGLq1IP%2F%2B8UcDSEBrVqpIuCuYl3Q7j7YaYZxk%3D&reserved=0
4,Flavobacterium poyangense,"[MH392710, JAALLN000000000]","was first isolated,, JXAS1T (=GDMCC 1.1378T=KCTC 62719T",https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006416%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7Cede9d3e056284e72eba408dc8c64635f%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638539610385722640%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=Qzrd94rIG9jH9V1Y7Vio8KHqnJN2A%2BWTRMYxBHLt780%3D&reserved=0
5,Mangrovimonas cancribranchiae,"[OR538380, CP136925]",", UG2_1T (=KCTC 102158T=DSM 117025T",https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006415%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7Cede9d3e056284e72eba408dc8c64635f%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638539610385733829%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=9pz15TOQ5qI7YT5FdfiMS5qM0vdHP45L1RRtx%2BhrDUo%3D&reserved=0
6,Clostridium aquiflavi,"[OR414528, JAVJAN000000000]",", 5 N-1T (=CICC 24886T=JCM 35355T",https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006420%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7Cede9d3e056284e72eba408dc8c64635f%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638539610385746457%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiL

In [25]:
pub2_df = pub_df.explode(['Accessions']).reset_index(drop=True)
pub2_df

,PublishedName,Accessions,Strains,URL
0,[Roseateles subflavus ],JASVDS000000000,is APW6T (=KACC 22877T=TBRC 16606T,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006426%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7Cede9d3e056284e72eba408dc8c64635f%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638539610385701322%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=3kRQU%2Bl1KWXJyM%2B7l1yPL2Ex%2FzlEC06BZdrpv28Vgt8%3D&reserved=0
1,[Roseateles subflavus ],OQ799017,is APW6T (=KACC 22877T=TBRC 16606T,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006426%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7Cede9d3e056284e72eba408dc8c64635f%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638539610385701322%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=3kRQU%2Bl1KWXJyM%2B7l1yPL2Ex%2FzlEC06BZdrpv28Vgt8%3D&reserved=0
2,[Roseateles aquae ],JAVXZY000000000,is APW11T (=KACC 22878T=TBRC 16607T,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006426%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7Cede9d3e056284e72eba408dc8c64635f%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638539610385701322%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=3kRQU%2Bl1KWXJyM%2B7l1yPL2Ex%2FzlEC06BZdrpv28Vgt8%3D&reserved=0
3,[Roseateles aquae ],OP341190,is APW11T (=KACC 22878T=TBRC 16607T,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006426%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7Cede9d3e056284e72eba408dc8c64635f%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638539610385701322%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=3kRQU%2Bl1KWXJyM%2B7l1yPL2Ex%2FzlEC06BZdrpv28Vgt8%3D&reserved=0
4,[Roseateles violae ],JAUHHC000000000,is PFR6T (=KACC 23257T=TBRC 17653T,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006426%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7Cede9d3e056284e72eba408dc8c64635f%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638539610385701322%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=3kRQU%2Bl1KWXJyM%2B7l1yPL2Ex%2FzlEC06BZdrpv28Vgt8%3D&reserved=0
5,[Roseateles violae ],OQ799020,is PFR6T (=KACC 23257T=TBRC 17653T,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006426%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7Cede9d3e056284e72eba408dc8c64635f%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638539610385701322%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=3kRQU%2Bl1KWXJyM%2B7l1yPL2Ex%2FzlEC06BZdrpv28Vgt8%3D&reserved=0
6,[Paenibacillus hexagrammi ],OM319628,of Paenibacillus hexagrammi is YPD9-1T (=KCTC 43424T =LMG 32988T,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006419%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7Cede9d3e056284e72eba408dc8c64635f%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638539610385711878%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=H5hpjVJGLq1IP%2F%2B8UcDSEBrVqpIuCuYl3Q7j7YaYZxk%3D&reserved=0
7,[Paeni

In [ ]:
#pub3_df = pub_df.explode(['PublishedName']).reset_index(drop=True)
#pub3_df

In [26]:
pub4_df = pub2_df.explode(['PublishedName']).reset_index(drop=True)
pub4_df.rename(columns={'Accessions' : 'accession'}, inplace=True)
pub4_df

,PublishedName,accession,Strains,URL
0,Roseateles subflavus,JASVDS000000000,is APW6T (=KACC 22877T=TBRC 16606T,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006426%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7Cede9d3e056284e72eba408dc8c64635f%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638539610385701322%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=3kRQU%2Bl1KWXJyM%2B7l1yPL2Ex%2FzlEC06BZdrpv28Vgt8%3D&reserved=0
1,Roseateles subflavus,OQ799017,is APW6T (=KACC 22877T=TBRC 16606T,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006426%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7Cede9d3e056284e72eba408dc8c64635f%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638539610385701322%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=3kRQU%2Bl1KWXJyM%2B7l1yPL2Ex%2FzlEC06BZdrpv28Vgt8%3D&reserved=0
2,Roseateles aquae,JAVXZY000000000,is APW11T (=KACC 22878T=TBRC 16607T,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006426%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7Cede9d3e056284e72eba408dc8c64635f%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638539610385701322%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=3kRQU%2Bl1KWXJyM%2B7l1yPL2Ex%2FzlEC06BZdrpv28Vgt8%3D&reserved=0
3,Roseateles aquae,OP341190,is APW11T (=KACC 22878T=TBRC 16607T,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006426%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7Cede9d3e056284e72eba408dc8c64635f%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638539610385701322%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=3kRQU%2Bl1KWXJyM%2B7l1yPL2Ex%2FzlEC06BZdrpv28Vgt8%3D&reserved=0
4,Roseateles violae,JAUHHC000000000,is PFR6T (=KACC 23257T=TBRC 17653T,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006426%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7Cede9d3e056284e72eba408dc8c64635f%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638539610385701322%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=3kRQU%2Bl1KWXJyM%2B7l1yPL2Ex%2FzlEC06BZdrpv28Vgt8%3D&reserved=0
5,Roseateles violae,OQ799020,is PFR6T (=KACC 23257T=TBRC 17653T,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006426%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7Cede9d3e056284e72eba408dc8c64635f%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638539610385701322%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=3kRQU%2Bl1KWXJyM%2B7l1yPL2Ex%2FzlEC06BZdrpv28Vgt8%3D&reserved=0
6,Paenibacillus hexagrammi,OM319628,of Paenibacillus hexagrammi is YPD9-1T (=KCTC 43424T =LMG 32988T,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006419%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7Cede9d3e056284e72eba408dc8c64635f%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638539610385711878%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=H5hpjVJGLq1IP%2F%2B8UcDSEBrVqpIuCuYl3Q7j7YaYZxk%3D&reserved=0
7,Paenibacillus hexagrammi,CP0

In [ ]:
#df_unique = pub4_df.drop_duplicates(["accession"], keep="first")
#df_unique

### Create a dataframe of unique accessions for srcchk

In [ ]:
#looks like I can skip this cell
#df_unique = pub4_df.drop_duplicates(["accession"], keep="first")
#df_unique.loc['Strains'] = (pd.DataFrame(df_unique.Strains.tolist())
#                      .fillna('')
#                      .astype(str)
#                      .agg(','.join, 1)
#                      .str.strip(','))
#df_unique

In [ ]:
#pub4_df = pub4_df.dropna()
#pub4_df

In [ ]:
#df_unique.dtypes

In [ ]:
#df_unique = df_unique.apply(lambda x: pd.Series(x.dropna().values))
#df_unique = df_unique.fillna('')

In [27]:
pub4_df['accession'] = pub4_df['accession'].astype('str') 

In [28]:
with open('acclist', 'w') as f:
    for text in pub4_df['accession'].tolist():
        f.write(text + '\n')



In [29]:
os.system("/netopt/ncbi_tools64/bin/srcchk -i acclist -f taxname,taxid,strain -o acclist.taxdata")


0

In [30]:
taxdata_file_name = (r'acclist.taxdata')    
srcchk_df = pd.read_csv(taxdata_file_name, sep='\t', index_col=None, low_memory=False)
srcchk_df.drop(columns=['Unnamed: 4'], inplace=True)
srcchk_df.rename(columns={'organism' : 'NCBIname'}, inplace=True)
srcchk_df['accession'] = srcchk_df['accession'].astype(str).replace('\.\d+', '', regex=True).astype(str)
srcchk_df = srcchk_df.dropna()
srcchk_df 
#many need to add dropna here

,accession,NCBIname,taxid,strain
0,JASVDS000000000,Pelomonas sp. APW6,3053353,APW6
1,OQ799017,Roseateles subflavus,3053353,APW6
2,JAVXZY000000000,Paucibacter sp. APW11,3077235,APW11
3,OP341190,Roseateles aquae,3077235,APW11
4,JAUHHC000000000,Pelomonas sp. PFR6,3058042,PFR6
5,OQ799020,Roseateles violae,3058042,PFR6
6,OM319628,Paenibacillus hexagrammi,2908839,YPD9-1
7,CP090978,Paenibacillus hexagrammi,2908839,YPD9-1
8,CP090979,Paenibacillus hexagrammi,2908839,YPD9-1
9,MH392710,Flavobacterium sp. JXAS1,2204302,JXAS1


In [31]:
combine_df=pd.merge(left=pub4_df, right=srcchk_df, left_on='accession', right_on='accession', how = 'outer')
combine_df = combine_df[['PublishedName', 'NCBIname', 'Strains', 'accession', 'strain', 'taxid', 'URL' ]]
combine_df

,PublishedName,NCBIname,Strains,accession,strain,taxid,URL
0,Roseateles subflavus,Pelomonas sp. APW6,is APW6T (=KACC 22877T=TBRC 16606T,JASVDS000000000,APW6,3053353,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006426%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7Cede9d3e056284e72eba408dc8c64635f%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638539610385701322%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=3kRQU%2Bl1KWXJyM%2B7l1yPL2Ex%2FzlEC06BZdrpv28Vgt8%3D&reserved=0
1,Roseateles subflavus,Roseateles subflavus,is APW6T (=KACC 22877T=TBRC 16606T,OQ799017,APW6,3053353,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006426%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7Cede9d3e056284e72eba408dc8c64635f%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638539610385701322%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=3kRQU%2Bl1KWXJyM%2B7l1yPL2Ex%2FzlEC06BZdrpv28Vgt8%3D&reserved=0
2,Roseateles aquae,Paucibacter sp. APW11,is APW11T (=KACC 22878T=TBRC 16607T,JAVXZY000000000,APW11,3077235,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006426%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7Cede9d3e056284e72eba408dc8c64635f%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638539610385701322%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=3kRQU%2Bl1KWXJyM%2B7l1yPL2Ex%2FzlEC06BZdrpv28Vgt8%3D&reserved=0
3,Roseateles aquae,Roseateles aquae,is APW11T (=KACC 22878T=TBRC 16607T,OP341190,APW11,3077235,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006426%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7Cede9d3e056284e72eba408dc8c64635f%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638539610385701322%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=3kRQU%2Bl1KWXJyM%2B7l1yPL2Ex%2FzlEC06BZdrpv28Vgt8%3D&reserved=0
4,Roseateles violae,Pelomonas sp. PFR6,is PFR6T (=KACC 23257T=TBRC 17653T,JAUHHC000000000,PFR6,3058042,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006426%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7Cede9d3e056284e72eba408dc8c64635f%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638539610385701322%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=3kRQU%2Bl1KWXJyM%2B7l1yPL2Ex%2FzlEC06BZdrpv28Vgt8%3D&reserved=0
5,Roseateles violae,Roseateles violae,is PFR6T (=KACC 23257T=TBRC 17653T,OQ799020,PFR6,3058042,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006426%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7Cede9d3e056284e72eba408dc8c64635f%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638539610385701322%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=3kRQU%2Bl1KWXJyM%2B7l1yPL2Ex%2FzlEC06BZdrpv28Vgt8%3D&reserved=0
6,Paenibacillus hexagrammi,Paenibacillus hexagrammi,of Paenibacillus hexagrammi is YPD9-1T (=KCTC 43424T =LMG 32988T,OM319628,YPD9-1,2908839,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006419%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7Cede9d3e056284e72eba408dc8c64635f%7C14b7757897

In [ ]:
combine_df['boo'] = combine_df['PublishedName'] != combine_df['NCBIname']

combine_df.style.applymap('background-color: red', subset=['PublishedName'])

combine_df

In [ ]:
 def color_positive_green(val):
    if combine_df.iloc['boo'] == True:
        color = 'red'
    else:
        color = 'black'
    return 'color: %s' % color
 
combine_df.style.applymap(color_positive_green)

In [ ]:
combine_df.apply_style_by_indexes(combine_df[combine_df['PublishedName'] != combine_df['NCBIname']], styler_obj=Styler(bg_color='red'), cols_to_style=['PublishedName', 'NCBIname'])

In [32]:
combine_df.to_excel('NameCheck.xlsx', engine='xlsxwriter', index = False) 

In [ ]:
#df.style.apply(color, axis=None).to_excel('styled.xlsx', engine='openpyxl')

### taxonerd test

In [ ]:
from taxonerd import TaxoNERD
taxonerd = TaxoNERD(prefer_gpu=False)
nlp = taxonerd.load(model="en_ner_eco_md", exclude=[], linker="taxref", threshold=0.7)
nlp.pipe_names

### option to use biopython to retrieve orgnames based on accessions

In [ ]:
import Bio
from Bio import Entrez
from Bio import SeqIO

In [ ]:
handle = Entrez.efetch(db="nucleotide", id="AY851612", rettype="gb", retmode="text")
x = SeqIO.read(handle, 'genbank')
x.annotations['organism']